# Predicting Fuel Theft 


During one of my past projects, a specific issue was detecting fuel theft from the vehicles under our surveillance. We had access to end-of-day fuel levels, idle time, and on-time data, but unfortunately, we did not have continuous access to fuel level readings. Monitoring for abrupt changes in fuel level would have been a simpler approach if we had such data at our disposal.


## Variables

We want to produce a method of detecting deviations from our fuel use prediction and actual consumption.


Given use time $a$ and average fuel consumption when in use $\alpha$;  idle time $b$ and average fuel consumption when idle $\beta$ gives us a predicted fuel use of $ p = a\alpha + b\beta$ and we want to plot against real fuel consumption $r$

## Data set

Unfortunatly I do not have access to our dataset so I will attempt to construct my own. 

We will treat idle fuel consumption as 4 litres per hour + random noise. And usage fuel consumption as 13 litres per hour + noise.

Due to variations in load of activities it is include a random noise of larger variation $\mu$.  i.e driving, digging, etc. Fortunately fuel theft only occurs while vehical would be at rest so we do not need to account for the large deviations of driving large distances as we will only select days where the vehical would be on site. We will have $\nu$. So we will have,

$p_0 =  a(\alpha + \mu_0)+ b(\beta +\nu_0)$


We will assume $\mu$ and $\nu$ are normally distributed with a mean of 0 and variance ${\sigma_{\alpha}}^2$ and ${\sigma_{\beta}}^2$

For the sake of this example I will chose ${\sigma_{\beta}}^2 = 2{\sigma_{\alpha}}^2$

We will save this in a Pandas DataFrame and use Plotly to visualise this.
Recently I have been using plotly over matlibplot as it seems to offer a richer suite for data visualisation.


In [52]:
import pandas as pd
import chart_studio.plotly as plotly
import plotly.offline  as pyoff
import plotly.graph_objects as go
import numpy as np


df = pd.DataFrame({"a":[],"mu":[],"b":[],"nu":[], "t":[], "fuelUse":[], "pred":[]})

alpha = 13
beta =4
sigma = 0.02
vehicalCount = 1000


## Constructing an appropriate PDF for time

We will use a left-skewed Weibul Distribution to model working hours. With a small tale ofter 8 hours.

The Weibull Probability Distribution is defined as:

$f = (\frac{k}{\lambda})(\frac{x}{\lambda})^(k-1) \exp(-(\frac{x}{\lambda})^k)$

We have $\lambda$ gives the location of where $x|_{f_{max}}$. We will chose a value just below 8, as typical working day is 8 hours, minus breaks. So we will choose $\lambda=6$

We expect it to be more common to have work less than more than 6 hours. So we want a small right tail and larger left tail. Graphically, if we choose $10\le k\le20$ we have a distribution that seems reasonable.


In [53]:
_lambda = 6
_k = 10



x = np.linspace(0, 10, 100)
xprime = x/_lambda


def pdf(x, l, k):
    f = (k/l)*((x/l)**(k-1))*(np.exp(-(x/l)**(k)))
    return f


y = pdf(x, _lambda, _k)


def getTrace(x, y, title):
    trace = go.Scatter(
        x=x,
        y=y,
        mode='lines',
        name=title,
        marker=dict(
            color='#C54C82'
        )
    )

    layout = go.Layout(
        title=title,
        showlegend=True
    )

    return [trace]


pdftrace = getTrace(x, y, "Weibull Dist")

fig = go.Figure(pdftrace)

pyoff.iplot(fig)


## Inverse Transform Sampling

Now we have an appropriate PDF, how do you create a random sample that fits this distribution.

Lets say we have a generic pdf $f(x), x \in \mathbb{R}$ with CDF $F$, and a uniform sample of $u, u\in[0,1]$

From $u$ we want to find the smallest $x$ such that $F(x) \ge u$, we take the infinum of x because for a generic equation it is not guranteed that we have a unique inverse.

To find our $x$ we will take $F^{-1}(u) = x$


In [54]:
from scipy import interpolate

import plotly.express as px
x = np.linspace(0, 24, vehicalCount)


def pdf(x, l, k):
    f = (k/l)*((x/l)**(k-1))*(np.exp(-(x/l)**(k)))
    return f


def cdf(pdf):
    return np.cumsum(pdf)


def icdf(cdf):
    idcf = interpolate.interp1d(cdf[0], cdf[1])
    return icdf


linSamples = np.random.uniform(0, 24, len(x))
cdf1 = cdf(pdf(x, 6, 10))


arr = np.array(vehicalCount)
arr = interpolate.interp1d(cdf1, x, fill_value="extrapolate")(linSamples)


fig = px.histogram(arr,
                   nbins=50)
pyoff.iplot(fig)


In [55]:
df["t"] = arr

df["a"] = df["t"]*np.random.rand()

df["b"] = df["t"]-df["a"]
df["mu"] = np.random.normal(0, 2*sigma, vehicalCount)
df["nu"] = np.random.normal(0, sigma, vehicalCount)

df["fuelUse"] = alpha*(df["a"]+df["mu"]) + beta*(df["b"]+df["nu"])
df["pred"] = alpha*(df["a"]) + beta*(df["b"])



## Refining our dataset

We actually expect the variation from the mean to increase as time increases. This would lead to a larger variation from the mean as time increases. To do this we will introduced constants $\zeta$ such that our error terms will instead be $\zeta \alpha \mu$ and $\frac{\zeta}{4} \beta \nu $, thus error increases over time and is proportional to fuel use.


In [56]:
zeta =0.15

df["mu"] = df["mu"] * zeta *alpha
df["nu"] = df["nu"] * zeta/4  *beta

df["fuelUse"] = alpha*(df["a"]+df["mu"]) + beta*(df["b"]+df["nu"])


fig = px.scatter(x=df["t"],y=df["fuelUse"])

fig.show()

## Adding Stolen Fuel

Now we have constructed our dataset we now want to add instances of stolen fuel.

Lets have a range between 2 and 15 litres stolen in 2% of the vehicals

In [57]:
stolenInstances = round(vehicalCount*0.02)


for i in range(stolenInstances):
    index = np.random.randint(0, 99)
    randar = np.random.uniform(2, 15)
    df["fuelUse"].iloc[index] = df["fuelUse"].iloc[index] - randar


fig = px.scatter(x=df["pred"], y=df["fuelUse"])
fig.show()


## Line of best fit with Simple Linear Regression

Obviously we can analytically work out the function but thats a little boring and introduces no new interesting theory so we will use linear regression. 
As the graph shows, and we expected. We have a very strong linear relationship between our prediction and actual fuel consumption. Suspiciouus activity visually deviates from this line. We want to have some form of "rank" or "suspicion index" based on this deviation.

First we want to find the line that the data fits on. We will do this with simple linear regression.

We will define the position vector of our datapoints as $\begin{pmatrix} pred \\ real \end{pmatrix}$

Consider a linear function $y = \alpha + \beta x$.

Given $n$ pairs of data  $\begin{pmatrix} x_i \\ y_i \end{pmatrix}$. We expect this to be described by $y_i = \alpha + \beta x_i +\epsilon_i$ with a error term $\epsilon_i$.

Consider $\sum_{i=1}^{n} {\epsilon_i}^2 = \sum_{i=1}^{n} (y_i - \alpha - \beta x_i)^2$

We want to find values $\hat{\alpha},\hat{\beta}$ that minimise this sum. So we have the smallest error term. 

For $\hat{\alpha}$, the y intercept. This makes sense to just be $\bar{y} - (\hat{\beta}\bar{x})$

For the gradient we have $\hat{\beta} = \frac{\sum_{i=1}^{n}(x_i - \bar{x})(y_i - \bar{y})}{\sum_{i=1}^{n}(x_i - \bar{x})^2}$

It can be shown that $\hat{\beta}=\frac{ \text{cov}(x,y)}{\text{var}(x)}$

Lets plot this:

In [58]:
var_x = np.var(df["fuelUse"])
cov_xy = np.cov(df["fuelUse"], df["pred"])[0][1]

ybar = np.average(df["pred"])
xbar = np.average(df["fuelUse"])
x = np.linspace(0,max(df["fuelUse"])+0.5,50)

beta = cov_xy/var_x

alpha  = ybar - (beta*xbar)


y = alpha + beta*x


df["Score"] = abs(df["fuelUse"] - (alpha + beta*df["pred"]))
df["Suspicious"] = np.nan
std_x = np.std(df["Score"])



for i in range(len(df["Score"])):
   if (df["Score"].iloc[i]>(3*std_x)):   
      df["Suspicious"].iloc[i] = df["fuelUse"].iloc[i]

fig = go.Figure()
fig = px.scatter(df, x="fuelUse",y="pred", color="Score")

fig.add_scatter(x=x,y=y)

In [59]:
def GenerateSample(alpha=13, beta=4, sigma=0.02, vehicalCount=1000, _lambda=6, _k=10, zeta=0.15):
    df = pd.DataFrame({"a": [], "mu": [], "b": [], "nu": [],
                      "t": [], "fuelUse": [], "pred": []})

    def pdf(x, l, k):
        f = (k/l)*((x/l)**(k-1))*(np.exp(-(x/l)**(k)))
        return f

    x = np.linspace(0, 10, vehicalCount)

    def cdf(pdf):
        return np.cumsum(pdf)

    linSamples = np.random.uniform(0, 24, len(x))
    cdf1 = cdf(pdf(x, _lambda, _k))

    idcf = np.array(vehicalCount)
    idcf = interpolate.interp1d(cdf1, x, fill_value="extrapolate")(linSamples)

    df["t"] = idcf

    df["a"] = df["t"]*np.random.rand()

    df["b"] = df["t"]-df["a"]
    df["mu"] = np.random.normal(0, 2*sigma, vehicalCount) * zeta * alpha
    df["nu"] = np.random.normal(0, sigma, vehicalCount) * zeta/4 * beta

    df["fuelUse"] = alpha*(df["a"]+df["mu"]) + beta*(df["b"]+df["nu"])
    df["pred"] = alpha*(df["a"]) + beta*(df["b"])
    stolenInstances = round(vehicalCount*0.02)

    for i in range(stolenInstances):
        index = np.random.randint(0, 99)
        randar = np.random.uniform(2, 15)
        df["fuelUse"].iloc[index] = df["fuelUse"].iloc[index] - randar

    return df


def SimpleLinearRegression(df):
    var_x = np.var(df["fuelUse"])
    cov_xy = np.cov(df["fuelUse"], df["pred"])[0][1]

    ybar = np.average(df["pred"])
    xbar = np.average(df["fuelUse"])
    x = np.linspace(0, max(df["fuelUse"])+0.5, 50)

    beta = cov_xy/var_x

    alpha = ybar - (beta*xbar)

    y = alpha + beta*x

    df["Score"] = abs(df["fuelUse"] - (alpha + beta*df["pred"]))
    df["Suspicious"] = np.nan
    std_x = np.std(df["Score"])

    for i in range(len(df["Score"])):
        if (df["Score"].iloc[i] > (3*std_x)):
            df["Suspicious"].iloc[i] = df["fuelUse"].iloc[i]

    fig = go.Figure()
    fig = px.scatter(df, x="fuelUse", y="pred", color="Score")

    fig.add_scatter(x=x, y=y)
    fig.show()


## Flaws in this analysis

I used this as a quick way to identify suspicious incidences however we notice variance increases overtime. So if $\zeta$ was large enough. The standard deviation of data could be too large to identify outliers over the whole dataset. That is, if the variance at $T\gg 0$ is very large, our outliers even tho, visually shifted from the line, may not be significant at a lower $T$.

In part 2 we will try to find a different method of analysis to solve this flaw.

In [60]:
df = GenerateSample(zeta=0.5)

SimpleLinearRegression(df)
